In [1]:
import pandas as pd
from openpyxl import load_workbook
import openpyxl
import numpy as np
from datetime import date, timedelta

In [2]:
def cell_to_num(char):
    char = char.upper()
    if char.isalpha():
        num = ord(char) - 64
    else:
        num = 0
    return num
print(cell_to_num('a'))

1


In [3]:
"""
# Windows file system
first_file = 'C:/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = 'C:/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'
"""
# linux file system
first_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report.xlsx'
second_file = '/mnt/c/Users/panka/Downloads/Nozzle Sales Report (1).xlsx'

wb_check = load_workbook(first_file)

/home/pankaj/.local/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [4]:
def data_cleaning(path):
    wb = load_workbook(path)
    ws = wb.active
    
    #Unmerge cell
    ws.unmerge_cells('O1:P1')
    ws.unmerge_cells('Q1:R1')
    ws.unmerge_cells('S1:T1')
    ws.unmerge_cells('U1:V1')
    if path[-7] == '1':
        #renaming issue_sale value with dates
        ws['S1'] = str(date.today() - timedelta(days=4))
        ws['T1'] = str(date.today() - timedelta(days=3))
    else:
        ws['S1'] = str(date.today() - timedelta(days=2))    
        ws['T1'] = str(date.today() - timedelta(days=1))
    
    print(ws['S1'].value)
    print(ws['T1'].value)    
    
    #Delete column U:V
    ws.delete_cols(cell_to_num('U'), cell_to_num('V')-cell_to_num('U')+1)
    #Delete column O:R
    ws.delete_cols(cell_to_num('O'), cell_to_num('R')-cell_to_num('O')+1)
    #Delete column K:M
    ws.delete_cols(cell_to_num('K'), cell_to_num('M')-cell_to_num('K')+1)
    #Delete column B:F
    ws.delete_cols(cell_to_num('B'), cell_to_num('F')-cell_to_num('B')+1)
    #Delete second row
    ws.delete_rows(2)
    wb.save(path)

In [5]:
def create_pivot(path):
    #reading excel file using pandas
    old_df = pd.read_excel(path)
    current_wb = load_workbook(path)
    ws = current_wb.active
    
    #filter out the 'NA' value in the product column
    df = old_df[
        (old_df['Product'] == 'XP') |
        (old_df['Product'] == 'HS') |
        (old_df['Product'] == 'MS')
    ]
    
    df.loc[df['Product'] == 'XP', 'Product'] = 'MS'
    
    pvt_all = pd.pivot_table(df,
                            index=["SA"],
                            columns=['Product'],
                            values=[ws['H1'].value, ws['G1'].value],
                            aggfunc=np.sum)
    return pvt_all

In [6]:
data_cleaning(first_file)
data_cleaning(second_file)

2020-08-11
2020-08-12
2020-08-09
2020-08-10


In [7]:
pvt = pd.concat([create_pivot(first_file), create_pivot(second_file)], axis=1)
print(pvt)

/home/pankaj/.local/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


                      2020-08-11           2020-08-12           2020-08-09  \
Product                       HS        MS         HS        MS         HS   
SA                                                                           
Fuels- Bargarh(R)      47651.560  35819.91   52833.13  53278.24   46489.12   
Fuels-Bhawanipatna(R   64470.060  46552.05   59586.83  44825.88   60822.19   
Fuels-JEYPORE(R)       96491.660  61671.44   81674.94  56544.08   67152.60   
Fuels-Jharsuguda(R)   125234.868  40942.03  121961.41  41689.80  148258.80   
Fuels-ROURKELA(R)     364201.220  49499.99  308070.64  51913.55  267085.81   
Fuels-SAMBALPUR (R)    75210.150  34169.23   91054.45  36489.29   64707.41   
Keonjhar I RSA        128017.360  25376.09  142144.60  25838.19   92381.44   
Keonjhar II RSA        11414.480   6577.45   11240.14   8027.82    5678.02   

                               2020-08-10            
Product                     MS         HS        MS  
SA                               

In [8]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    pvt.to_excel(writer, 'pivot sheet', index=True)

In [9]:
df = pd.read_excel(first_file, sheet_name='pivot sheet')
print(df.columns.values)

['Unnamed: 0' '2020-08-11' 'Unnamed: 2' '2020-08-12' 'Unnamed: 4'
 '2020-08-09' 'Unnamed: 6' '2020-08-10' 'Unnamed: 8']


In [10]:
cols = df.columns.values
order = [0, 3, 4, 1, 2, 7, 8, 5, 6]
cols = [cols[i] for i in order]
print(cols)

['Unnamed: 0', '2020-08-12', 'Unnamed: 4', '2020-08-11', 'Unnamed: 2', '2020-08-10', 'Unnamed: 8', '2020-08-09', 'Unnamed: 6']


In [11]:
df = df[cols]
print(df)

             Unnamed: 0 2020-08-12 Unnamed: 4 2020-08-11 Unnamed: 2  \
0               Product         HS         MS         HS         MS   
1                    SA        NaN        NaN        NaN        NaN   
2     Fuels- Bargarh(R)    52833.1    53278.2    47651.6    35819.9   
3  Fuels-Bhawanipatna(R    59586.8    44825.9    64470.1    46552.1   
4      Fuels-JEYPORE(R)    81674.9    56544.1    96491.7    61671.4   
5   Fuels-Jharsuguda(R)     121961    41689.8     125235      40942   
6     Fuels-ROURKELA(R)     308071    51913.6     364201      49500   
7   Fuels-SAMBALPUR (R)    91054.5    36489.3    75210.1    34169.2   
8        Keonjhar I RSA     142145    25838.2     128017    25376.1   
9       Keonjhar II RSA    11240.1    8027.82    11414.5    6577.45   

  2020-08-10 Unnamed: 8 2020-08-09 Unnamed: 6  
0         HS         MS         HS         MS  
1        NaN        NaN        NaN        NaN  
2    52161.7      39691    46489.1    32735.8  
3    54229.5    49233.8   

In [12]:
with pd.ExcelWriter(first_file, engine='openpyxl') as writer:
    writer.book = load_workbook(first_file)
    df.to_excel(writer, 'summary', index=False)

In [13]:
wb = load_workbook(first_file)
ws = wb['summary']
ws.delete_rows(3)
check = 0
for value in ws.iter_rows(min_row=1, max_row=1):
    for cell in value:
        if check % 2 == 0:
            cell.value = ""
        check += 1
        print(cell.value)
# print(df.columns.values)
print(ws['A3'].value)


2020-08-12

2020-08-11

2020-08-10

2020-08-09

Fuels- Bargarh(R)


In [14]:
"""
data = 'ABCDEFGHI'
for char in data:
    val = char + '1'
    print(ws[val].value)

print(ws.cell(row=1, column=2).value)

#Iterate through columns of the first row
for row in ws.iter_rows(min_row=1, max_row=1):
    for cell in row:
        print(cell.value)
"""

"\ndata = 'ABCDEFGHI'\nfor char in data:\n    val = char + '1'\n    print(ws[val].value)\n\nprint(ws.cell(row=1, column=2).value)\n\n#Iterate through columns of the first row\nfor row in ws.iter_rows(min_row=1, max_row=1):\n    for cell in row:\n        print(cell.value)\n"

In [15]:
from exchangelib import Account, Message, Credentials, HTMLBody
from exchangelib import Configuration, DELEGATE
import os

In [16]:
outlook_user = os.environ.get('OUTLOOK_USER')
outlook_password = os.environ.get('OUTLOOK_PASS')
outlook_server = os.environ.get('OUTLOOK_SERVER')
outlook_email = os.environ.get('OUTLOOK_EMAIL')

print(os.getenv('OUTLOOK_USER'))
print(os.getenv('OUTLOOK_PASS'))

ioc\00504802
Toptal@22


In [17]:
credentials = Credentials(username=outlook_user,
                         password=outlook_password
                         )
config = Configuration(server=outlook_server,
                      credentials=credentials)
account = Account(primary_smtp_address=outlook_email,
                 config=config,
                 autodiscover=False,
                 access_type=DELEGATE)

In [18]:
def create_html(ws):
    html = """
    <html><body><h3>Nozzle Sales Report:</h3>
    <table border=1>
    """
    # entering header data in html
    for whole_row in ws.iter_rows(min_row=1, max_row=2):
        html += "<tr>"
        for row in whole_row:
            html += f'<th>{row.value}</th>'
        html += "</tr>"
    # entering table data in html
    for whole_row in ws.iter_rows(min_row=3):
        html += "<tr>"
        for row in whole_row:
            if isinstance(row.value, float):
                html += f'<td>{round(row.value, 2)}</td>'
                print(round(row.value, 2))
            else:
                html += f'<td>{row.value}</td>'
        html += "</tr>"
    html += """
    </table>
    <p>
    <br>
    With Regards<br>
    Pankaj Barnwal<br>
    </p>
    </body>
    </html>
    """
    return html

In [19]:
msg = Message(
    account=account,
    subject="Nozzle Sales Report - Sambalpur DO",
    body=HTMLBody(create_html(ws)),
    to_recipients=['barnwalp@indianoil.in']
)

msg.send_and_save()

52833.13
53278.24
47651.56
35819.91
52161.72
39691.02
46489.12
32735.83
59586.83
44825.88
64470.06
46552.05
54229.5
49233.8
60822.19
39487.42
81674.94
56544.08
96491.66
61671.44
92905.92
69497.63
67152.6
44219.26
121961.41
41689.8
125234.87
40942.03
168201.78
46338.04
148258.8
26295.15
308070.64
51913.55
364201.22
49499.99
329813.85
58321.39
267085.81
31631.72
91054.45
36489.29
75210.15
34169.23
80230.1
39320.62
64707.41
20116.5
142144.6
25838.19
128017.36
25376.09
97599.26
28058.12
92381.44
14858.24
11240.14
8027.82
11414.48
6577.45
9387.91
34524.5
5678.02
5250.26
